In [13]:
from sympy import *
from sympy.vector import CoordSys3D, Del

In [30]:
R = CoordSys3D('R')
delop = Del()
x = R.x
y = R.y

eps, theta = symbols('epsilon theta')
nx = Function('n_x')(x, y)
ny = Function('n_y')(x, y)

In [31]:
n = nx * R.i + ny * R.j
n

(n_x(R.x, R.y))*R.i + (n_y(R.x, R.y))*R.j

### Calculate charge density

In [42]:
splay = simplify( delop.dot(n).doit()**2 )
splay

(Derivative(n_x(R.x, R.y), R.x) + Derivative(n_y(R.x, R.y), R.y))**2

In [46]:
bend = simplify( n.cross( delop.cross(n).doit() ).magnitude()**2 )
bend = bend.subs(nx**2 + ny**2, 1)
bend

(Derivative(n_x(R.x, R.y), R.y) - Derivative(n_y(R.x, R.y), R.x))**2

In [47]:
g = (1 + eps) * splay + (1 - eps) * bend
g

(1 - epsilon)*(Derivative(n_x(R.x, R.y), R.y) - Derivative(n_y(R.x, R.y), R.x))**2 + (epsilon + 1)*(Derivative(n_x(R.x, R.y), R.x) + Derivative(n_y(R.x, R.y), R.y))**2

### Calculate in terms of director angle

In [48]:
phi = Function('phi')(x, y)
phi

phi(R.x, R.y)

In [57]:
g = simplify( g.subs({nx: cos(phi), ny: sin(phi)}) )
g

(1 - epsilon)*(sin(phi(R.x, R.y))*Derivative(phi(R.x, R.y), R.y) + cos(phi(R.x, R.y))*Derivative(phi(R.x, R.y), R.x))**2 + (epsilon + 1)*(sin(phi(R.x, R.y))*Derivative(phi(R.x, R.y), R.x) - cos(phi(R.x, R.y))*Derivative(phi(R.x, R.y), R.y))**2

### Plug it into [Euler-Lagrange equation](https://www.wikiwand.com/en/Euler%E2%80%93Lagrange_equation#Single_function_of_several_variables_with_single_derivative)

In [66]:
euler_lagrange = simplify(g.diff(phi) - (g.diff(phi.diff(x)).diff(x) + g.diff(phi.diff(y)).diff(y)))
euler_lagrange = trigsimp(factor(euler_lagrange, eps))
display(euler_lagrange)

2*epsilon*(-2*sin(phi(R.x, R.y))**2*Derivative(phi(R.x, R.y), R.x)*Derivative(phi(R.x, R.y), R.y) - sin(phi(R.x, R.y))**2*Derivative(phi(R.x, R.y), (R.x, 2)) + sin(phi(R.x, R.y))**2*Derivative(phi(R.x, R.y), (R.y, 2)) - sin(2*phi(R.x, R.y))*Derivative(phi(R.x, R.y), R.x)**2 + sin(2*phi(R.x, R.y))*Derivative(phi(R.x, R.y), R.y)**2 + 2*sin(2*phi(R.x, R.y))*Derivative(phi(R.x, R.y), R.x, R.y) + 2*cos(phi(R.x, R.y))**2*Derivative(phi(R.x, R.y), R.x)*Derivative(phi(R.x, R.y), R.y) + cos(phi(R.x, R.y))**2*Derivative(phi(R.x, R.y), (R.x, 2)) - cos(phi(R.x, R.y))**2*Derivative(phi(R.x, R.y), (R.y, 2))) - 2*sin(phi(R.x, R.y))**2*Derivative(phi(R.x, R.y), (R.x, 2)) - 2*sin(phi(R.x, R.y))**2*Derivative(phi(R.x, R.y), (R.y, 2)) - 2*cos(phi(R.x, R.y))**2*Derivative(phi(R.x, R.y), (R.x, 2)) - 2*cos(phi(R.x, R.y))**2*Derivative(phi(R.x, R.y), (R.y, 2))